## Offline Notebook for Fibre Identification

This notebook can be used to run the fibre identification code outside the actor, for testing and development. 

There are three options shown

- run with a FITS file as input (performs the centroiding)
- run with a dataframe containing the information read from the database as input
- run by querying the database directly


The variable targetPos is a numpy array of length nx3 where 
- 0: cobraID
- 1: target x position
- 2: target y position

If the cobras are parked at the centre, centrePos can be used; it can also be used for more general cases but can cause mis-identifications. 


In [ ]:
%matplotlib notebook
import mcsActor.windowedCentroid.centroid as centroid
import mcsActor.mcsRoutines.mcsRoutines as mcsTools
from astropy.io import fits
import matplotlib.pylab as plt
from importlib import reload
from pfs.utils import butler
import pandas as pd
import pfs.utils.coordinates.transform as transformUtils
import numpy as np
import offlineID as offline # this has wrappers to call the various routines and format things
import os


## setup paths and variables

Change the data path to the directory where the fits files are (if needed), and choose the appropriate camera. 

In [ ]:
dataPath = "/Users/karr/Science/Data/PFS/convergeTests/20220222_009/data/"

# camera (canon for on telescope, rmod for test stand)
cameraName = 'rmod_71m'
#cameraName = 'canon_50m'

# get the geometry
bbutler = butler.Butler()
pfi = bbutler.get("moduleXml", moduleName="ALL", version="")
dots = bbutler.get("black_dots", moduleName="ALL", version="")
fids = bbutler.get('fiducials')

# variables for IDing
centrePos, armLength, dotPos, goodIdx, des = mcsTools.readCobraGeometry(pfi,dots)
adjacentCobras = mcsTools.makeAdjacentList(centrePos, armLength)
centParms = mcsTools.getCentroidParams([])
fidPos =  np.array([fids['fiducialId'],fids['x_mm'],fids['y_mm']]).T


## FITS file input 

Pass a frame ID, the routine reads the telescope information from the FITS header,
does the centroiding, matches and returns the following numpy arrays 

cobraMatch of of length(ncobras) where
- 0: cobraID
- 1: spotID (in centroids)
- 2: x_mm
- 3: y_mm

fidMatch
- 0: fiducialID
- 1: spotID (in centroids)
- 2: x_mm
- 3: y_mm

centroids
- 0: spotID
- 1: x_pixel
- 2: y_pixel
- 3 -> other spot shape information 


In [ ]:
#version for fits files

# list of frameIDs
frameID = 7612200
targetPos = centrePos

cobraMatch,fidMatch,centroids=offline.runMatchFile(frameID,dataPath,centParms,cameraName,fids,centrePos,armLength,dotPos,goodIdx,adjacentCobras,targetPos)


## CSV file file input 

expects two pandas arrays as input, one with the mcs_exposure data, one with the mcs_data data. 

returns the same as above



In [ ]:
# version for query from database dumped to csv (mcs_exposure and mcs_data)
reload(offline)
# list of frameIDs
frameID = 7612200

expData = pd.read_csv("/Users/karr/mcsExposure_7612200.csv")
mcsData = pd.read_csv("/Users/karr/mcsData_7612200.csv")

cobraMatch,fidMatch,centroids=offline.runMatchDF(frameID,mcsData,expData,cameraName,fids,centrePos,armLength,dotPos,goodIdx,adjacentCobras,centrePos)


### Database Input

Similar to CSV input, but reads the exposure and centroid data from the database. Requires db object with link to the database. 

Returns the same as above

In [ ]:
# version for DB
frameID = 7612200

sql = """select * from mcs_exposure where mcs_frame_id=7612200"""
expData = db.fetch_query(sql)
sql = """select * from mcs_data where mcs_frame_id=7612200"""
mcsData = db.fetch_query(sql)

cobraMatch,fidMatch,centroids=offline.runMatchDF(frameID,mcsData,centroidData,cameraName,fids,centrePos,armLength,dotPos,goodIdx,adjacentCobras,centrePos)




## Quick Plots

Does a quick plots of the matching, with each spot connected to it's matched cobra centre position. 
Fiducial fibres and patrol regions are marked. 


In [ ]:
fig,ax=plt.subplots()
for i in range(len(goodIdx)):
    ax.plot([cobraMatch[i,2],des.centers.real[goodIdx[i]]],[cobraMatch[i,3],des.centers.imag[goodIdx[i]]])
ax.scatter(centrePos[:,1],centrePos[:,2])
ax.scatter(fids['x_mm'],fids['y_mm'],marker="d")
for i in range(len(centrePos)):
    circle=plt.Circle((centrePos[i,1],centrePos[i,2]),armLength[i],fill=False,color='black')
    a=ax.add_artist(circle)

